In [ ]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem, oedocking, oeomega
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

In [ ]:
from openeye import oechem, oedocking, oeomega
# Generate multi conformers from smile strings
imsfname = 'test.ism'
omega = oeomega.OEOmega()
omega.SetMaxConfs(1)
with oechem.oemolistream(imsfname) as ifs:
    for mol in ifs.GetOEMols():
        with oechem.oemolostream('lig_oebs/{}_isomers.oeb.gz'.format(mol.GetTitle())) as ofs:
            for enantiomer in oeomega.OEFlipper(mol.GetActive(), 12, True):
                enantiomer = oechem.OEMol(enantiomer)
                if omega(enantiomer):
                    oechem.OEWriteMolecule(ofs, enantiomer)
                    

In [6]:
from openeye import oechem, oedocking, oeomega
#Read in the Receptor
receptor = oechem.OEGraphMol()
apofname = 'epoxide_hydrolase_apo_receptor.oeb.gz'
if not oedocking.OEReadReceptorFile(receptor, apofname):
    OEThrow.Fatal("Unable to read receptor")

#Initialize Docking
dock_method = oedocking.OEDockMethod_Hybrid
if not oedocking.OEReceptorHasBoundLigand(receptor):
    oechem.OEThrow.Warning("No bound ligand, switching OEDockMethod to ChemGauss4.")
    dock_method = oedocking.OEDockMethod_Chemgauss4
dock_resolution = oedocking.OESearchResolution_Default
dock = oedocking.OEDock(dock_method, dock_resolution)
sdtag = oedocking.OEDockMethodGetName(dock_method)

In [5]:
#Parse from smile strings
imsfname = 'test.ism'
omega = oeomega.OEOmega()
omega.SetMaxConfs(3)
with oechem.oemolistream(imsfname) as ifs:
    for mol in ifs.GetOEMols():
        with oechem.oemolostream('lig_oebs/{}_dock.oeb.gz'.format(mol.GetTitle())) as ofs:
            if omega(mol):
                print("Generating {} conformers for {}".format(omega.GetMaxConfs(), mol.GetTitle()))
                if dock.Initialize(receptor):
                    print("Docking", mol.GetTitle())
                    dockedMol = oechem.OEMol()
                    dock.DockMultiConformerMolecule(dockedMol, mol)
                    oedocking.OESetSDScore(dockedMol, dock, sdtag)
                    dock.AnnotatePose(dockedMol)
                    oechem.OEWriteMolecule(ofs, dockedMol)
                    print("\tTotal score = %f" % dock.ScoreLigand(dockedMol))
                    print("\tScore components contributions to score:")
                    for comp in dock.GetComponentNames():
                        print("\t%12s: %6.2f" % (comp, dock.ScoreLigandComponent(dockedMol, comp)))     

Generating 1 conformers for JF6_1
Docking JF6_1
	Total score = -7.782097
	Score components contributions to score:
	  CG3:Steric:  -9.31
	   CG3:Clash:   0.19
	CG3:ProDesolv:   0.57
	CG3:LigDesolv:   2.23
	CG3:LigDesolvHB:  -0.23
	      CG4:HB:  -1.22
Generating 1 conformers for 4VY_1
Docking 4VY_1
	Total score = -7.929983
	Score components contributions to score:
	  CG3:Steric: -10.45
	   CG3:Clash:   0.11
	CG3:ProDesolv:   2.35
	CG3:LigDesolv:   0.92
	CG3:LigDesolvHB:  -0.12
	      CG4:HB:  -0.74
Generating 1 conformers for BSU_1
Docking BSU_1
	Total score = -9.562876
	Score components contributions to score:
	  CG3:Steric:  -8.86
	   CG3:Clash:   0.81
	CG3:ProDesolv:   2.03
	CG3:LigDesolv:   2.70
	CG3:LigDesolvHB:  -3.24
	      CG4:HB:  -3.00
